missing deps
- deepspeed
- plotly
- nbformat

In [ ]:
import re
import tqdm
import numpy as np
import pandas as pd
import plotly.express as px
from functools import reduce
from openminers.text_to_text.dummy.test_template import TestMiner

In [2]:
miner = TestMiner()
for _ in range(5):
    print('---'*30,'\n',miner.model(''))

------------------------------------------------------------------------------------------ 
 Interesting question.
In order to address that context it's probably useful to tentatively medidate on the contextual viewpoints of the context.
------------------------------------------------------------------------------------------ 
 I see your point
In order to appropriately answer that context it's probably intelligent to simply medidate on the subjective value of the context.
------------------------------------------------------------------------------------------ 
 Well now then..
In order to appropriately answer that question it's terribly valuable to first examine the subjective utility of the question.
------------------------------------------------------------------------------------------ 
 I've been waiting for this moment.
In order to nail that query it's positively important to simply ruminate over the spiritual essence of the query.
-------------------------------------------

In [ ]:
nunique = reduce(lambda x,y: x*y, [len(words) for words in miner.choices.values()],1)
print(f'Number of unique completions is {nunique:,}')

In [ ]:
def substr_freq_reward(completion, sub='a'):
    # this reward model loves the letter a
    return completion.count(sub)/len(completion)

def word_length_reward(completion):
    return np.mean([len(word) for word in completion.split()])*0.5


miner = TestMiner()
miner.learning_rate=0.5

messages = [
    {'role':'system', 'content':'You are a good boy'},
    {'role':'user', 'content':'What is the capital of Texas?'}
]
results = []
scores = []
responses = []
max_no_change = 10
pbar = tqdm.tqdm(range(10000))
for i in pbar:
    # print('--'*40)
    response = miner.forward(messages=messages)
    responses.append(response)
    if i>max_no_change and len(set(responses[-max_no_change:]))==1:
        print(f'Response has not changed for {max_no_change} steps... quitting training')
        break
    
    rewards = substr_freq_reward(response, sub='at')    
    # rewards = word_length_reward(response)
    # print(f'response = {response!r}')
    # print(f'rewards = {rewards:.3f}')
    miner.backward(messages=messages, response=response, rewards=rewards)
    scores.append(rewards)
    if i%10 == 0:
        pbar.set_description(f'Rewards: {rewards:.3f}')
    
    for key, words in miner.choices.items():
        results.append(pd.DataFrame({'step':i, 'response':response, 'rewards':rewards, 'key':key, 'words':words,'weights':miner.weights[key]}))

px.line(pd.Series(scores, name='Reward')).show()
# miner.weights


In [ ]:
df = pd.concat(results)

for key in miner.choices:
    df_sub = df.loc[df['key']==key].copy()
    df_sub['weights_moving_avg'] = df_sub.weights.rolling(5).mean()
    px.line(df_sub,x='step',y='weights_moving_avg',color='words',
            title=key, template='plotly_white',
            ).update_traces(opacity=0.4).show()

In [ ]:
miner = TestMiner()

In [ ]:
nunique = reduce(lambda x,y: x*y, [len(words) for words in miner.choices.values()],1)
print(f'Number of unique completions is {nunique:,}')

In [ ]:
for i in range(3):
    print(f'{"---"*30}\n{i}:\n{miner.model("")}')

In [ ]:
completion = miner.model("")
completion

In [ ]:
miner.template

In [ ]:
response = miner.forward(messages=messages)
response

In [ ]:
cc = '('+'|'.join(miner.choices['opening_phrase'])+')'
cc = cc.replace('.','\.')
cc

In [ ]:
re.search(cc, response)

In [ ]:
keywords = re.findall(r"{(.*?)}", miner.template)
print(keywords)

pattern = re.escape(miner.template)
seen = set()
for i, key in enumerate(keywords):
    if key not in seen:
        name = key
        seen.add(key)
    else:
        name = f'key{i}'    
    wordmatch = ('('+'|'.join(miner.choices[key])+')').replace('.','\.')
    pattern = pattern.replace("\\{"+key+"\\}", f"(?P<{name}>{wordmatch})",1)
pattern


In [ ]:
completion = "Yes, that's a good question\nIn order to appropriately answer that query it's possibly wise to simply be cognizant of the intended perspectives of the query."
# Use the pattern to extract the values
match = re.match(pattern, completion)
selected_words = match.groupdict()
selected_words

In [ ]:
choice_indices = {key: words.index(selected_words[key]) for key, words in miner.choices.items()}
choice_indices

In [ ]:
rewards = 10.
for key, index in choice_indices.items():
    miner.weights[key][index] += miner.learning_rate * rewards
    miner.weights[key]/=miner.weights[key].sum()